In [72]:
import pandas as pd
from collections import defaultdict
import re

# Define the file path
file_path = '../datas/Option_A/Supply and demand series_Time_by_rows.xlsx'

# Read the Excel file
excel_file = pd.ExcelFile(file_path)

# Get the sheet name you want to process
sheet_name = '1.Argentina'  # Replace with the actual sheet name

# Read the selected sheet into a DataFrame
df = pd.read_excel(excel_file, sheet_name)

# Define the activities and country_activities with shortened names
activities = ['Production', 'Import', 'Export', 'Stock change', 'Unused',
              'Total supply', 'Refineries', 'Power plants', 'Self-producers', 'Gas plants',
              'Charcoal plants', 'Coke plants and blast furnaces', 'Distilleries', 'Other centers',
              'Total transformation', 'Own consumption', 'Losses', 'Adjustment', 'Transport',
              'Industrial', 'Residential', 'Commercial, services, public',
              'Agriculture, fishing and mining', 'Construction and others',
              'Energy consumption', 'Non-energy consumption', 'Final consumption']


country="Argentina"
country_activities = [f'{country} - {activity}' for activity in activities]

# Initialize the dictionary to store tables
tables = defaultdict(list)

# Loop through each row and distribute tables based on keywords
current_table = None
for row in df.itertuples(index=False):
    if isinstance(row[0], str):  # Assuming the table names are in the first column
        # Check if the row contains a keyword for a new table
        for k in activities:
            if k in row[0]:
                current_table = k[:31]  # Truncate the table name to 31 characters
                tables[current_table].append(row)
    elif current_table and pd.isnull(row[0]):
        # Empty row encountered, treat as table delimiter
        current_table = None
    elif current_table:
        # Append rows to the current table
        tables[current_table].append(row)

# Write each table to a new sheet in the output file
output_file = 'output.xlsx'  # Replace with your desired output file path
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    for table_name, table_rows in tables.items():
        # Create a DataFrame for the table
        table_df = pd.DataFrame(table_rows)

        # Set the table headers as column names
        table_df.columns = table_df.iloc[0]

        # Drop the first row (header)
        table_df = table_df.iloc[1:].reset_index(drop=True)

        # Modify the table name using regex to include only the activity name
        activity_name = re.search(r'\b(?:{})\b'.format('|'.join(activities)), table_name, flags=re.IGNORECASE)
        if activity_name:
            activity_name = activity_name.group(0)
        else:
            activity_name = 'Table'

        # Write the table to a new sheet in the output file
        table_df.to_excel(writer, sheet_name=activity_name, index=False)

In [73]:
import re
import pandas as pd
from collections import defaultdict
import openpyxl

# Define the file path
file_path = '../datas/Option_A/Supply and demand series_Time_by_rows.xlsx'

# Read the Excel file
excel_file = pd.ExcelFile(file_path)

# Get the sheet name you want to process
sheet_name = '1.Argentina'  # Replace with the actual sheet name

# Read the selected sheet into a DataFrame
df = pd.read_excel(excel_file, sheet_name)

# Define the activities and country_activities with shortened names
activities = ['Production', 'Import', 'Export', 'Stock change', 'Unused',
              'Total supply', 'Refineries', 'Power plants', 'Self-producers', 'Gas plants',
              'Charcoal plants', 'Coke plants and blast furnaces', 'Distilleries', 'Other centers',
              'Total transformation', 'Own consumption', 'Losses', 'Adjustment', 'Transport',
              'Industrial', 'Residential', 'Commercial, services, public',
              'Agriculture, fishing and mining', 'Construction and others',
              'Energy consumption', 'Non-energy consumption', 'Final consumption']

country_activities = [f'{country} - {activity}' for activity in activities]

# Initialize the dictionary to store tables
tables = defaultdict(list)

# Loop through each row and distribute tables based on keywords
current_table = None
for row in df.itertuples(index=False):
    if isinstance(row[0], str):  # Assuming the table names are in the first column
        # Check if the row contains a keyword for a new table
        for k in country_activities:
            if k in row[0]:
                current_table = k[:31]  # Truncate the table name to 31 characters
                tables[current_table].append(row)
    elif current_table and pd.isnull(row[0]):
        # Empty row encountered, treat as table delimiter
        current_table = None
    elif current_table:
        # Append rows to the current table
        tables[current_table].append(row)

# Create a new workbook
workbook = openpyxl.Workbook()

# Write each non-empty table to a new sheet in the workbook
for table_name, table_rows in tables.items():
    if table_rows:  # Check if the table has rows
        # Create a sheet for the table
        sheet = workbook.create_sheet(title=table_name)

        # Write the table rows to the sheet
        for row in table_rows:
            sheet.append(row)

# Save the workbook
output_file = 'output.xlsx'  # Replace with your desired output file path
workbook.save(output_file)

In [74]:
len(activities)

27